In [1]:
import numpy as np
import pandas as pd
import shutil
import skimage.io
from sklearn.ensemble import BaggingClassifier
import os
import skimage.transform
from sklearn.neural_network import MLPClassifier
from os import listdir
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
from natsort import natsorted
import cv2
import os
import h5py
import matplotlib.pyplot as plt 
from skimage.feature import hog
import warnings
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.externals import joblib

/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [33]:
labels = pd.read_csv("./bonus-sml-2020/SML_Train.csv") # seprating files class wise

# Create `train_sep` directory
train_dir='./bonus-sml-2020/SML_Train/'
train_sep_dir='./bonus-sml-2020/Train_Sep/'
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)

for filename,class_name in labels.values:
    # Create subdirectory with `class_name`
    if not os.path.exists(train_sep_dir+str(class_name)):
        os.mkdir(train_sep_dir+str(class_name))
    src_path=train_dir+filename
    dst_path=train_sep_dir+str(class_name)+'/'+ filename
    shutil.copy(src_path,dst_path)

In [2]:
def imageAugmntation(image,count,target_dir): #vertica flip of images
    #flip
    img=cv2.imread(image) 
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    vertical_img = img.copy()
    vertical_img = cv2.flip( img, 1 )
    #print(target_dir+'/'+'Train_'+str(count)+'.jpg')
    cv2.imwrite(target_dir+'/'+'Train_'+str(count)+'.jpg',cv2.cvtColor(vertical_img, cv2.COLOR_BGR2RGB))

In [3]:
def Gaussinablur(image,count,target_dir): #gaussina blur of imgaes
    img = cv2.imread(image) 
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    blur=cv2.GaussianBlur(img,(9,9),0)
    print(target_dir+'/'+'Train_'+str(count)+'.jpg')
    cv2.imwrite(target_dir+'/'+'Train_'+str(count)+'.jpg',cv2.cvtColor(vertical_img, cv2.COLOR_BGR2RGB))

In [4]:
size=tuple((500, 500))
train_path="./bonus-sml-2020/Train_sep/"
test_path="./bonus-sml-2020/SML_Test/"
h5_data='./bonus-sml-2020/output/data.h5'
h5_labels='./bonus-sml-2020/output/labels.h5'
bins=8
seed=9
scoring="accuracy"
test_size=0.10
num_trees=700

In [5]:
train_labels = os.listdir(train_path)
print(train_labels)
train_labels = list(map(int, train_labels))
train_labels.sort()
print(train_labels)
globalFeat = []
labels=[]

['20', '18', '9', '0', '11', '7', '16', '6', '17', '1', '10', '19', '8', '21', '24', '23', '4', '15', '3', '12', '2', '13', '5', '14', '22']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [38]:
for training_name in train_labels: #data balancing
    # join the training data path and each species training folder
    dir = os.path.join(train_path, str(training_name))
    # loop over the images in each sub-folder
    count=0
    for f in os.listdir(dir):
        if not f.startswith('.'):
            count+=1
    
    file_name=16000
    for file in listdir(dir):
        if not file.startswith('.'):
            if(count<=982):
                imageAugmntation(dir+'/'+file,file_name,dir)
                file_name+=1
                count+=1
    print(count)

983
983
972
983
983
983
896
908
983
983
983
896
983
983
983
983
983
983
983
983
983
983
983
983
983


In [6]:
def hogD(image,hog_pixels_per_cell=64,hog_cells_per_block=1):# hog features
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    hog_desriptor=hog(gray,orientations=8,pixels_per_cell=(hog_pixels_per_cell,hog_pixels_per_cell),cells_per_block=(hog_cells_per_block, hog_cells_per_block),feature_vector=True)
    return hog_desriptor

In [7]:
def hu_momentsF(image):# hu_moments features
    image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    feature=cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

In [8]:
def haralickF(image): #haralick feature
    gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    haralick=mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [9]:
def histogramF(image, mask=None): #histogram featurs
    image=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    hist=cv2.calcHist([image],[0, 1, 2],None,[bins, bins, bins],[0, 256, 0, 256, 0, 256])
    cv2.normalize(hist,hist)
    return hist.flatten()

In [10]:
training_data_feature_map={} #combining all features generated
daisy_descriptor_list=[]
for training_name in train_labels:
    dir=os.path.join(train_path, str(training_name))
    label=training_name
    for file in listdir(dir):
        image=cv2.imread(dir+'/'+file)
        image=cv2.resize(image,size)
        haralick= haralickF(image)
        hu_moments=hu_momentsF(image)
        histogram=histogramF(image)
        hog_des=hogD(image)
        globalF=np.hstack([histogram, haralick, hu_moments,hog_des])
        #globalF=np.hstack([histogram, haralick, hu_moments])
        labels.append(label)
        globalFeat.append(globalF)

    print(len(globalFeat[0]))

print("Done")

924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
924
Done


In [11]:
print(np.array(globalFeat).shape)
print(np.array(labels).shape)
target=labels
print(np.array(target).shape)
scaler=MinMaxScaler(feature_range=(0, 1))
scaled=scaler.fit_transform(globalFeat)
h5f_data = h5py.File(h5_data, 'w')
h5f_data.create_dataset('dataset_1',data=np.array(scaled))
h5f_label = h5py.File(h5_labels, 'w')
h5f_label.create_dataset('dataset_1',data=np.array(target))
h5f_data.close()
h5f_label.close()

(24315, 924)
(24315,)
(24315,)


In [12]:
models = []
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_depth=50,random_state=0,min_samples_split=2)))
print(models)
results = []
names   = []
h5f_data  = h5py.File(h5_data, 'r')
h5f_label = h5py.File(h5_labels, 'r')
globalFeat_string = h5f_data['dataset_1']
global_labels_string   = h5f_label['dataset_1']
globalFeat=np.array(globalFeat_string)
global_labels=np.array(global_labels_string)
h5f_data.close()
h5f_label.close()
print(np.array(globalFeat).shape)
print(np.array(global_labels).shape)

[('RF', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False))]
(24315, 924)
(24315,)


In [13]:
# split the training and testing data
(X_train_S,X_test_S,y_train_S,y_test_S)=train_test_split(np.array(globalFeat),np.array(global_labels),test_size=test_size,random_state=seed)
print("Train data ",X_train_S.shape)
print("Test data",X_test_S.shape)
print("Train labels",y_train_S.shape)
print("Test labels",y_test_S.shape)                                                         

Train data  (21883, 924)
Test data (2432, 924)
Train labels (21883,)
Test labels (2432,)


In [14]:
for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed)
    cv_results = cross_val_score(model,X_train_S,y_train_S,cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    print(cv_results.mean())

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.5532148026153705


In [15]:
globalFeat_test=[]
for file in natsorted(listdir(test_path)):
        image = cv2.imread(test_path+file)
        image = cv2.resize(image,size)
        hu_moments=hu_momentsF(image)
        haralick=haralickF(image)
        histogram=histogramF(image)
        hog_des=hogD(image)
        globalF = np.hstack([histogram,haralick,hu_moments,hog_des])
        #globalF=np.hstack([histogram,haralick,hu_moments])
        globalFeat_test.append(globalF)
print("Done")

Done


In [16]:
scaler=MinMaxScaler(feature_range=(0, 1))
scaled_test=scaler.fit_transform(globalFeat_test)
print(np.array(scaled_test).shape)

(1500, 924)


In [17]:
import matplotlib.pyplot as plt
clf=RandomForestClassifier(n_estimators=num_trees, max_depth=50,random_state=0,min_samples_split=2)
clf.fit(X_train_S,y_train_S)
y_test=clf.predict(scaled_test)

In [18]:
def helper(direc):
    files=[]
    for filename in natsorted(listdir(direc)):
        files.append(filename)
    return files

files=helper('./bonus-sml-2020/SML_Test/')
data_res=pd.DataFrame()
data_res['id']=files
data_res['category']=y_test
data_res.to_csv (r'sample_submission.csv', index = False, header=True)

In [19]:
# dump model 
import joblib
filename = './bonus-sml-2020/MT19124_model/Best_3/RF_Best3.sav'

joblib.dump(clf, filename)

['./bonus-sml-2020/MT19124_model/Best_3/RF_Best3.sav']